In [ ]:
# MCP Database Analysis Engine - Quick Feature Test
# All imports consolidated here
import pandas as pd
from datetime import datetime
from analysis_engine import AnalysisEngine

from config import RESORT_MAPPING

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("✅ All modules imported successfully!")


In [ ]:
# Configuration
OUTPUT_DIR = "reports"  # Output directory for Excel reports
resorts = RESORT_MAPPING  # All available resorts

print(f"📊 Configuration: {len(resorts)} resorts available")

In [ ]:
# Batch report generation for all resorts
analysisEngine = AnalysisEngine(OUTPUT_DIR)
saved_files = []

for i, resort_config in enumerate(resorts, 1):
    resort_name = resort_config.get('resortName')
    print(f"[{i}/{len(resorts)}] Processing {resort_name}...")
    try:
        result = analysisEngine.generate_analysis(resort_config=resort_config, run_date=datetime.now(), analysis_type="rep")
        if result.get('report_path'):
            saved_files.append(result['report_path'])
            print(f"✅ Success!")
    except Exception as e:
        print(f"❌ Failed: {e}")

print(f"\n✨ Generated {len(saved_files)}/{len(resorts)} reports")


In [ ]:
# Single resort report generation (modify config as needed)
resort_configs = [
    {
        "dbName": "snowbowl",
        "resortName": "snowbowl",
        "groupNum": -1
    },
    {
        "dbName": "purgatory",
        "resortName": "purgatory",
        "groupNum": 46
    },
    {
        "dbName": "Nordic",
        "resortName": "Nordic",
        "groupNum": -1
    },
    {
        "dbName": "MCP",
        "resortName": "SANDIA",
        "groupNum": 10
    },
]

analysisEngine = AnalysisEngine(OUTPUT_DIR)

for config in resort_configs:
    print(f"|Generating reports for {config.get('resortName', 'unknown')}")

    result = analysisEngine.generate_analysis(
        resort_config=config,
        run_date="1/25/2026",
        analysis_type="rep",
        debug=True
    )

    print(
        f"✅ Report generated for {config.get('resortName', 'unknown')}: "
        f"{result.get('report_path', 'N/A')}"
    )

In [ ]:
# Comparison Insights Generator - Comparative analysis between two dates
analysisEngine = AnalysisEngine('insights')
print(f"✅ Analysis Engine initialized for comparison insights")

# Generate insights: Compare two dates (comparison vs anchor)
resort_config = {
    "dbName": "snowbowl",
    "resortName": "snowbowl",
    "groupNum": -1
}

comparison_date = "12/30/2025"  # MM/DD/YYYY
anchor_date = "12/30/2024"      # MM/DD/YYYY

print(f"🔍 Generating insights...")
print(f"   Resort: {resort_config['resortName']}")
print(f"   Comparison: {comparison_date} | Anchor: {anchor_date}\n")

try:
    results = analysisEngine.generate_comparison_insights(
        resort_config=resort_config,
        comparison_date=comparison_date,
        anchor_date=anchor_date,
        debug=True
    )
    
    print("✅ Insight Generation Complete!\n")
    
    # Analyze variance columns: Top 3 and Bottom 3 for each variance metric
    dept_df = results['department_analytics']
    if not dept_df.empty:
        variance_columns = [
            'Revenue Variance %',
            'Payroll Variance %',
            'Budget Variance %',
            'Revenue-to-Payroll Variance %',
            'Budget-to-Payroll Variance %'
        ]
        
        for var_col in variance_columns:
            if var_col in dept_df.columns:
                print(f"\n{'='*70}")
                print(f"📊 {var_col}")
                print(f"{'='*70}")
                
                # Top 3 (descending - highest variance)
                top3 = dept_df.nlargest(3, var_col)
                print(f"\n🔺 Top 3 (Highest Variance):")
                display_cols = ['Department Title', var_col, 'Comparison Revenue', 'Comparison Payroll']
                display(top3[display_cols])
                
                # Bottom 3 (ascending - lowest variance)
                bottom3 = dept_df.nsmallest(3, var_col)
                print(f"\n🔻 Bottom 3 (Lowest Variance):")
                display(bottom3[display_cols])
    
    # Visit analytics
    visit_df = results['visit_analytics']
    if not visit_df.empty:
        print(f"\n{'='*70}")
        print(f"🎫 Visit Variance Analysis")
        print(f"{'='*70}")
        if 'Visit Variance %' in visit_df.columns:
            top3_visits = visit_df.nlargest(3, 'Visit Variance %')
            bottom3_visits = visit_df.nsmallest(3, 'Visit Variance %')
            print(f"\n🔺 Top 3 (Highest Variance):")
            display(top3_visits[['Visit Category', 'Visit Variance %', 'Comparison Visits', 'Anchor Visits']])
            print(f"\n🔻 Bottom 3 (Lowest Variance):")
            display(bottom3_visits[['Visit Category', 'Visit Variance %', 'Comparison Visits', 'Anchor Visits']])
    
    print(f"\n💡 Debug files saved to: {analysisEngine.insights_dir}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
